# NumPy Universal Functions — Advanced Practice

This notebook contains **advanced (but not excessive)** practice problems focused on **NumPy universal functions (ufuncs)**.

Each problem includes:
- a **clear description** of the task and why it matters,
- a **fully vectorized solution** using NumPy best practices,
- and **sanity checks** (`assert`) to verify correctness.

Key topics covered:
- `out=` to reduce temporary allocations
- `where=` for safe conditional computation
- ufunc reductions (`reduce`, `accumulate`, `reduceat`)
- `at` for repeated indexed updates
- dtype casting pitfalls
- numerically stable computation patterns

In [1]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)

## Problem 1 — Column-wise Z-scores with `where=`

**Description**

Standardization (z-scoring) is a common preprocessing step in data analysis and machine learning.
Each feature (column) is transformed to have mean 0 and standard deviation 1:

\[
Z = (X - \mu) / \sigma
\]

This problem emphasizes:
- vectorized column-wise statistics,
- safe division using `where=`,
- handling edge cases where a column has zero variance.

**Task**
- Standardize each column of `X`.
- If a column has `sigma == 0`, keep that column all zeros.
- Do not use Python loops.

In [2]:
X = np.array(
    [[10, 1, 7],
     [12, 1, 3],
     [14, 1, 5],
     [16, 1, 9]],
    dtype=np.float64,
)

mu = X.mean(axis=0)
sigma = X.std(axis=0, ddof=0)
centered = X - mu

Z = np.zeros_like(centered)
np.divide(centered, sigma, out=Z, where=(sigma != 0))

print(Z)

assert np.allclose(Z[:, 1], 0.0)
assert np.allclose(Z[:, [0, 2]].mean(axis=0), 0.0)
assert np.allclose(Z[:, [0, 2]].std(axis=0, ddof=0), 1.0)

[[-1.3416  0.      0.4472]
 [-0.4472  0.     -1.3416]
 [ 0.4472  0.     -0.4472]
 [ 1.3416  0.      1.3416]]


## Problem 2 — Clamp Without `np.clip`

**Description**

Clamping values into a fixed interval is common in numerical pipelines
(e.g., bounding probabilities or sensor readings).

NumPy provides `np.clip`, but this exercise demonstrates that many operations
can be expressed by **composing simpler ufuncs**.

**Task**
- Clamp the values of `a` into the interval `[lo, hi]`.
- Use only ufuncs (`np.maximum`, `np.minimum`).

In [3]:
a = np.array([-3.5, -1.0, 0.0, 2.3, 9.9])
lo, hi = -1.0, 3.0

clamped = np.minimum(np.maximum(a, lo), hi)
print(clamped)

assert np.allclose(clamped, np.clip(a, lo, hi))

[-1.  -1.   0.   2.3  3. ]


## Problem 3 — Avoid Temporaries with `out=`

**Description**

Large intermediate arrays can significantly hurt performance and memory usage.
NumPy ufuncs allow you to reuse memory via the `out=` argument.

This problem highlights how a complex expression can be decomposed into
simple ufunc calls with reusable buffers.

**Task**
Compute:
\[
y = \frac{\sqrt{a^2 + b^2}}{1 + |a - b|}
\]

- Reuse arrays via `out=`.
- Verify correctness against a straightforward expression.

In [4]:
rng = np.random.default_rng(0)
a = rng.normal(size=2000)
b = rng.normal(size=2000)

num = np.empty_like(a)
tmp = np.empty_like(a)
den = np.empty_like(a)

np.multiply(a, a, out=num)
np.multiply(b, b, out=tmp)
np.add(num, tmp, out=num)
np.sqrt(num, out=num)

np.subtract(a, b, out=den)
np.abs(den, out=den)
np.add(den, 1.0, out=den)

y = np.empty_like(a)
np.divide(num, den, out=y)

y_ref = np.sqrt(a**2 + b**2) / (1 + np.abs(a - b))
assert np.allclose(y, y_ref)

## Problem 4 — Safe `log(p)/q` Using `where=`

**Description**

Real-world numerical data often contains invalid values.
Blindly applying functions like `log` or division can introduce warnings,
NaNs, or infinities.

**Task**
Compute:
\[
r = \frac{\log(p)}{q}
\]

Rules:
- If `p <= 0`, result is `NaN`
- If `q == 0`, result is `NaN`
- Otherwise compute normally

Use `where=` to avoid invalid operations.

In [5]:
p = np.array([1.0, 0.5, 0.0, -2.0, 3.0])
q = np.array([2.0, 0.0, 1.0, 4.0, -1.0])

valid = (p > 0) & (q != 0)
r = np.full_like(p, np.nan)

logp = np.empty_like(p)
np.log(p, out=logp, where=valid)
np.divide(logp, q, out=r, where=valid)

print(r)
assert np.isnan(r[1]) and np.isnan(r[2]) and np.isnan(r[3])

[ 0.         nan     nan     nan -1.0986]


## Problem 5 — Piecewise Function Without Invalid `sqrt`

**Description**

Conditional logic is common in numerical code.
Using `np.where` incorrectly may still evaluate invalid branches
(e.g., `sqrt` of negative numbers).

**Task**
Compute a piecewise function:
- if `x < 0`: return `x^2`
- else: return `sqrt(x)`

Ensure `sqrt` is **never applied** to negative values by using `where=` with `out=`.

In [6]:
x = np.array([-4.0, -1.0, 0.0, 1.0, 9.0], dtype=float)

y = np.empty_like(x)

neg = x < 0

np.square(x, out=y, where=neg)
np.sqrt(x, out=y, where=~neg)

y_ref = np.empty_like(x)
np.square(x, out=y_ref, where=(x < 0))
np.sqrt(x, out=y_ref, where=(x >= 0))

assert np.allclose(y, y_ref)

print("x :", x)
print("y :", y)


x : [-4. -1.  0.  1.  9.]
y : [16.  1.  0.  1.  3.]


## Problem 6 — Rolling Sum (Window = 3) Using Ufunc Reduction

**Description**

Rolling window computations appear frequently in signal processing and finance.
This exercise shows how to express a rolling sum using array shifting
and a ufunc reduction.

**Task**
- Compute the sum of every consecutive 3-element window in `x`.
- Use vectorized slicing and `np.add.reduce`.

In [7]:
x = np.array([2, 1, 0, 3, 4, -1, 2])
X3 = np.vstack([x[:-2], x[1:-1], x[2:]])
s = np.add.reduce(X3, axis=0)

assert np.array_equal(s, [3, 4, 7, 6, 5])

## Problem 7 — Segment-wise Cumulative Product

**Description**

Financial returns are often computed in segments (e.g., after resets or rebalancing).
This problem demonstrates combining simple Python control flow with
`np.multiply.accumulate` for efficient segment-wise computation.

**Task**
- Compute a cumulative product of returns.
- Restart the accumulation whenever `reset` is `True`.

In [8]:
g = np.array([1.02, 0.99, 1.01, 0.97, 1.03, 1.01, 0.98])
reset = np.array([True, False, False, True, False, False, False])

seg_id = np.cumsum(reset)
eq = np.empty_like(g)

for sid in np.unique(seg_id):
    idx = np.where(seg_id == sid)[0]
    eq[idx] = np.multiply.accumulate(g[idx])

assert np.allclose(eq, [1.02, 1.0098, 1.019898, 0.97, 0.9991, 1.009091, 0.988909])

## Problem 8 — Repeated Index Accumulation with `np.add.at`

**Description**

Index-based aggregation is common (e.g., histograms, group-by operations).
When indices repeat, naive indexing updates are incorrect.

**Task**
- Accumulate values into bins defined by `idx`.
- Use `np.add.at` to handle repeated indices correctly.

In [9]:
v = np.array([10.0, 1.5, 2.0, 3.0, 4.5])
idx = np.array([2, 0, 2, 2, 1])

totals = np.zeros(4)
np.add.at(totals, idx, v)

assert np.allclose(totals, [1.5, 4.5, 15.0, 0.0])

## Problem 9 — Dtype Casting Trap with `out=`

**Description**

Ufuncs write results into the dtype of the `out` array.
If that dtype is integer, floating-point results will be truncated.

**Task**
- Demonstrate the truncation bug.
- Fix it by allocating the output with a floating dtype.

In [10]:
a = np.array([1, 2, 3, 4])
b = np.array([2, 2, 2, 2])

out_int = np.empty_like(a)
np.divide(a, b, out=out_int, casting="unsafe")

assert np.array_equal(out_int, [0, 1, 1, 2])


## Problem 10 — Numerically Stable Softmax (Row-wise)

**Description**

Softmax is widely used in machine learning.
A naive implementation can overflow for large values.

**Task**
- Implement a row-wise softmax.
- Use the standard stability trick: subtract the row-wise maximum.
- Verify that each row sums to 1.

In [11]:
X = np.array([[1000, 1001, 999], [1, 2, 3]], dtype=float)

row_max = np.max(X, axis=1, keepdims=True)
Z = X - row_max
E = np.exp(Z)
S = E / np.sum(E, axis=1, keepdims=True)

assert np.allclose(S.sum(axis=1), 1.0)

## Problem 11 — Row-wise L2 Normalization with `where=`

**Description**

Vector normalization is common in geometry and machine learning.
Zero vectors require special handling to avoid division by zero.

**Task**
- Normalize each row of `X` to unit L2 norm.
- Leave zero rows unchanged.
- Use `where=` to avoid invalid division.

In [12]:
X = np.array([[3, 4], [0, 0], [1, -1]], dtype=float)

norm = np.sqrt(np.sum(X**2, axis=1, keepdims=True))
Y = np.zeros_like(X)
np.divide(X, norm, out=Y, where=(norm != 0))

assert np.allclose(np.sqrt((Y[0]**2).sum()), 1.0)
assert np.allclose(Y[1], 0.0)

## Problem 12 — Group Sums with `reduceat`

**Description**

`reduceat` allows you to apply reductions to variable-length groups
defined by starting indices.

**Task**
- Given `x` and group start indices, compute the sum within each group.
- Use `np.add.reduceat`.

In [13]:
x = np.array([2, 1, 5, 10, -3, 7, 8, 1])
starts = np.array([0, 3, 5])

group_sums = np.add.reduceat(x, starts)
assert np.array_equal(group_sums, [8, 7, 16])